# Lab #2

In [ ]:
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from PIL import Image

## Camera

Let's start with displaying the output of our camera. Initialize the camera with 300x300 pixels and link it to the widget for live stream.

In [ ]:
camera = Camera.instance(width=300, height=300)


In [ ]:
# display live camera feed
image_widget = ipywidgets.Image()
cam_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

Now, let's capture an image with the camera and save it.

In [ ]:
with open('data/capture.jpg', 'wb') as f:
    f.write(image_widget.value)

cam_link.unlink()

# display image
image_widget = ipywidgets.Image()
image_widget.value = open('data/capture.jpg', 'rb').read()
display(image_widget)

## Exploration

In this example we'll show how you can classify objects with JetBot! We'll use a pre-trained neural network (Resnet18) that was trained on the ImageNet dataset to detect different common objects. These include:

- Mailbox (index 637)
- Cup (index 968)
- ...

First, we'll want to import the resnet18 class which takes the pre-trained weights as input.

In [ ]:
from torchvision.models import resnet18
from torchvision import transforms as T
import torch
from PIL import Image
import json

model = resnet18(pretrained=True). eval()

preprocess = T.Compose([
    T.Resize(256), 
    T.CenterCrop(224), 
    T.ToTensor(), 
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

classes = json.load(open('data/imagenet_labels.json'))

Then let's try with sample images:

In [ ]:
def classify_image(img):
    # preprocess the image
    batch = preprocess(img).unsqueeze(0)

    # perform inference 
    with torch.no_grad():
        pred = model(batch)
        prob = pred.softmax(dim=1)

    class_id = prob.argmax(dim=1).item()
    score = prob[0, class_id].item()

    # print the result
    print(f"{classes[class_id]}: {100 * score:.1f}%")
    return img

In [ ]:
# load the image 
img = Image.open('data/mailbox.jpg')

classify_image(img)

In [ ]:
# load the image 
img = Image.open('data/cup.png')

classify_image(img)

Now, let's run the model using our image.

In [ ]:
# load the image 
img = Image.open('data/capture.jpg')

classify_image(img)